<h2>Ставим библиотеку для работы с поиском opensearch-py </h2>
<ol>
<li>Запускаем докер образ с настройками
<li>Проверяем что экземпляр поиска доступен
</ol>

<textarea style=" min-width:500px; max-width:100%;min-height:150px;height:100%;width:100%;">
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org opensearch-py

docker run -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "OPENSEARCH_INITIAL_ADMIN_PASSWORD=sea12323>RCh" opensearchproject/opensearch:latest

curl https://localhost:9200 -ku admin:sea12323>RCh
</textarea>

In [39]:
from opensearchpy import OpenSearch

host = 'localhost'
port = 9200
auth = ('admin', 'sea12323>RCh') # For testing only. Don't store credentials in code.
#ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Optional client certificates if you don't want to use HTTP basic authentication.
# client_cert_path = '/full/path/to/client.pem'
# client_key_path = '/full/path/to/client-key.pem'

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    # client_cert = client_cert_path,
    # client_key = client_key_path,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    #ca_certs = ca_certs_path
)

#
# "russian_morphology",
#                            "english_morphology",
# Create an index with non-default settings.
index_name = 'python-test-index'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 4
    },
      "analysis": {
                "analyzer": {
                    "product_name_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "search_synonym",
                            "ru_stopwords"
                        ]
                    }
                },
                "filter": {
                    "search_synonym": {
                        "ignore_case": "true",
                        "type": "synonym",
                        "synonyms": [
                            "шуруповерт, отвертка, ров => ров",
                        ]
                    },
                    "ru_stopwords": {
                        "type": "stop",
                        "stopwords": "а,без,более,бы,был,была,были,было,быть,в,вам,вас,весь,во,вот,все,всего,всех,вы,где,да,даже,для,до,его,ее,если,есть,еще,же,за,здесь,и,из,или,им,их,к,как,ко,когда,кто,ли,либо,мне,может,мы,на,надо,наш,не,него,нее,нет,ни,них,но,ну,о,об,однако,он,она,они,оно,от,очень,по,под,при,с,со,так,также,такой,там,те,тем,то,того,тоже,той,только,том,ты,у,уже,хотя,чего,чей,чем,что,чтобы,чье,чья,эта,эти,это,я,a,an,and,are,as,at,be,but,by,for,if,in,into,is,it,no,not,of,on,or,such,that,the,their,then,there,these,they,this,to,was,will,with"
                    }
                }
            }
  },
    "mappings": {
        "properties": {
            "name": {
                "type": "text",
                "analyzer": "product_name_analyzer"
            },
            "code": {
                "type": "keyword"
            },
            "description": {
                "type": "text",
                "analyzer": "product_name_analyzer"
            },
            "brand": {
                "type": "keyword"
            }
        }
    }
}


response = client.indices.create(index_name, body=index_body)
print('\nCreating index:')
print(response)

# Add a document to the index.
document = {
  'title': 'отвертка Moneyball',
  'director': 'Bennett Miller шуруповерт ров',
  'year': '2011'
}
id = '1'

response = client.index(
    index = index_name,
    body = document,
    id = id,
    refresh = True
)

print('\nAdding document:')
print(response)

# Perform bulk operations

movies = '{ "index" : { "_index" : "my-dsl-index", "_id" : "2" } } \n { "title" : "отвертка Interstellar", "director" : "Christopher Nolan отвертка", "year" : "2014"} \n { "create" : { "_index" : "my-dsl-index", "_id" : "3" } } \n { "title" : "Star Trek Beyond отвертка", "director" : "Justin Lin", "year" : "2015"} \n { "update" : {"_id" : "3", "_index" : "my-dsl-index" } } \n { "doc" : {"year" : "2016"} }'

client.bulk(movies)

# Search for the document.
q = 'отвертка'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'director']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "director": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results:')
print(response)

# Delete the document.
response = client.delete(
    index = index_name,
    id = id
)

print('\nDeleting document:')
print(response)

# Delete the index.
response = client.indices.delete(
    index = index_name
)

print('\nDeleting index:')
print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test-index'}

Adding document:
{'_index': 'python-test-index', '_id': '1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}

Search results:
{'took': 2, 'timed_out': False, '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.5753642, 'hits': [{'_index': 'python-test-index', '_id': '1', '_score': 0.5753642, '_source': {'title': 'отвертка Moneyball', 'director': 'Bennett Miller шуруповерт ров', 'year': '2011'}, 'highlight': {'title': ['<strong>отвертка</strong> Moneyball']}}]}}

Deleting document:
{'_index': 'python-test-index', '_id': '1', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}

Deleting index:
{'acknowledged': True}


In [41]:
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

#print(findFiles('data/1/articles/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,?/|\}]{[=-)(*;:&^%$#@!'-АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя0987654321"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

i = 0

text_lines = {}
all_files = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/1/articles/*.txt'):
    file_name = os.path.splitext(os.path.basename(filename))[0]
    all_files.append(file_name)
    lines = readLines(filename)
    i += 1
    print (file_name, lines[0], "\n", " ".join(lines))
    if i>5:
        break
    text_lines[file_name] = lines

n_files = len(all_files)
print (n_files)

4_36 ПЛЕТНЕВУ см. Наброски послания о продолжении Евгения Онегина 
 ПЛЕТНЕВУ см. Наброски послания о продолжении Евгения Онегина
4_22 ПИСАТЬ Я НЕ УМЕЮ... 
 ПИСАТЬ Я НЕ УМЕЮ...          Шуточныи коллективныи экспромт, результат своего рода литературнои игры, где каждыи из участников по очереди записывает несколько строк, подхватывая поэтическую тему, заданную предшественниками. Пушкину принадлежат ст. 34 (Я дружбои пламенею, / Я дружбе верен стал  АПСС. Т. 2, кн. 1. С. 109). Стихи 12 записаны рукои К. Н. Батюшкова, 56  рукои А. А. Плещеева, 78  В. А. Жуковского. Жуковскому же, по-видимому, принадлежит и помета под текстом: 4 сентября 1817 Сарское Село. Записан на одном листке с подобным же коллективным экспромтом Вяземскому (Зачем, забывши славу...), вероятно, во время одного из визитов арзамасцев к жившему летом в Царском Селе Н. М. Карамзину.
3_17 ЛЮБОВЬ ОДНА  ВЕСЕЛЬЕ ЖИЗНИ ХЛАДНОИ... 
 ЛЮБОВЬ ОДНА  ВЕСЕЛЬЕ ЖИЗНИ ХЛАДНОИ...          Одна из элегии так называемого бакунинского цикла (с

In [19]:
n_files

6